# Analysis of Brandeis classes in pandas
Here we show how to use pandas to analyze the Brandeis course data stored in data/courses

Some questions we can try to answer
* which faculty have taught the most students over the Fall2004-Summer2021 interval
* which departments offer the most courses
* which departments have had the most students
* how have the enrollents by departments changed
* etc, as in PA01

We'll show how to use pivot tables and aggregation to answer some of these.

This notebook shows how to use aggregation with groupby, pivot tables, and how to create new columns by applying some function to other columns.


In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

First we read in the data from Fall 2004 to Summer 2021 with minimal info about each course

In [ ]:
df = pd.read_csv('data2/classes/courses.csv')
df

# exporting pandas data frames to json
Here we show how to export a data frame to json


In [ ]:
import json
result = df.to_json(orient="split")
parsed = json.loads(result)
z = json.dumps(parsed, indent=4) 
len(z)
jfile = open('data2/courses_2005-2020.json','w')
jfile.write(z)
jfile.close()

# Finding teachers with the most students
The easiest way to do this is with a groupby it shows the three stages of groupby
1. grouping of rows together based on the value in a column ('inst')
1. projection of those rows into a set of values (what value are we interested in) (enr)
1. aggregation of those values into a single value with some operator from arrays to numbers (sum)

The groupby feature has the following syntax
``` python
a = df.groupby(GROUPED_COLUMNS)
b = a[TARGET_COLUMN]
c = b.agg(AGGREGATORS)
```
Its use is to compile summary statistics. 

The groupby method partitions the rows into groups that have the same values on the specified columns.

We can then specify some Target columns and some aggregators (sum, np.mean, np.max, len) to produce a single value for all of the rows in each group.

Some examples will make it clear!



In [ ]:
#fac = df.groupby(['inst','subject'])['enr'].agg(sum)
#fac
fac = df.groupby(['inst','subj'])
fac = fac['enr'].agg([sum, np.mean, np.max,np.min,len])
fac = fac.sort_values(ascending=False,by='sum')
fac.head(40)



## Here is another example
We're finding the statistics about courses that Dan Perlman has taught, grouped by subject.

In [ ]:

df[ df['inst']=='Perlman, Dan' ].groupby('title')['enr'].agg([sum, len, np.mean])

## Sorting the Series
We can sort the values if we want and we can use .head(N) and .tail(N) to look at either end of the sorted series.

In [ ]:
fac.sort_values(by='sum',ascending=False)

In [ ]:
fac.head(1010).tail(10)

In [ ]:
df

# Finding departments (subj) with the most students per year
Let's do another example of aggregation, to sum the enrollments for each department over all years

In [ ]:
depts = df.groupby('subj')['enr'].agg(sum)
depts

It is more interesting to look at enrollments per year

In [ ]:
def div17(x):
    return x/17
depts = depts.apply(div17)
depts

In [ ]:
depts = depts.sort_values()
depts = depts[::-1]
depts.head(20)

# Restricting to a time interval
Let's do the same but just look for the past 4 years, i.e. from Fall2017 to Summer 2021 which are the codes 1173 to 1212.


In [ ]:
df2 = df[df['code']>=1193]
df2
#islater = df['code']>= 1173
#df[islater]

## aggregate by summing the enrollments for each subject

In [ ]:
depts = df2.groupby(['subj'])['enr'].agg(sum)
depts = depts.sort_values(ascending=False)
depts.head(20)
                     


# Time series
Now lets try to get time series data and plot the enrollments in a department by year.
Let's first add a new column giving the academic year, so Fall2004 and Spring 2005 and Summer2005 would all be 2004


In [ ]:
def code2year(code):
    ''' returns the academic year corresponding to a code'''
    code=str(code)
    year = int("20"+code[1:-1])
    if code[-1]!='3':
        year = year -1
    return year
print(code2year('1051'))



### the original data frame has no 'year' column

In [ ]:
df

### create a new column for the year
We can apply the code2year function to each of the integers in the 'code' column to get a new series,
and then assign that series to be a new column 'year' of the df dataframe

In [ ]:
df['year'] = df['code'].apply(code2year)
df

# Group by the year and subject
This creates a new Series with a multi-index containing both year and subject

In [ ]:
depts_enr = df.groupby(['subj','year'])['enr'].agg(sum)
depts_enr['COSI'].plot()
plt.axis([2004,2021,0,2500])
plt.grid()


In [ ]:
depts_enr = df.groupby(['year','subj'])['enr'].agg(sum)
depts_enr

# Reset to a single index
We can use the reset_index method to make the subj part of the index into a column


In [ ]:
depts_enr = depts_enr.reset_index(level=['subj'])
depts_enr

# Create a Pivot table of enrollments by department and year
Now we do the usual construction of a pivot table

In [ ]:
depts_pt =  pd.pivot_table(depts_enr, values='enr', index=["year"], columns=["subj"])
depts_pt

# Plot selected columns of the pivot table
We can now select a few columns and plot them!

In [ ]:
depts_pt[['BIOL','CHEM','COSI','MATH','PHYS','BCHM']].plot(figsize=(15,10))
plt.grid()
plt.title("STEM Enrollments from 2004-2020")
plt.axis([2000,2022,0,3500])
plt.savefig('images/stemenr.png')

In [ ]:
depts_pt.plot(figsize=(15,10))
plt.legend([])

## Most popular course at Brandeis

In [ ]:
df

In [ ]:
df2 = df[df['code']>1042]
courses = df2.groupby(['title'])['enr'].agg(sum)
courses.sort_values(inplace=True,ascending=False)
courses.head(10)

In [ ]:
fac = df.groupby('title')
fac = fac['enr'].agg([sum])
fac = fac.sort_values(ascending=False, by='sum').head(10)
fac

In [ ]:
popcourses = df.groupby('title')['enr'].agg([sum]).sort_values(ascending=False, by='sum').head(10)
popcourses
